In [50]:
import bs4
import json
import nltk
import time
import string
import pickle
import requests

In [25]:
ROOT_URL = 'https://en.wikipedia.org/'

In [26]:
stop_words = nltk.corpus.stopwords.words('english')

In [27]:
with open('articles.json', 'r') as infile:
    articles = json.load(infile)

In [28]:
def get_all_leaves(tree_dict):
    if 'children' not in tree_dict.keys():
        return [tree_dict['id']]
    leaves = []
    for child in tree_dict['children']:
        leaves.extend(get_all_leaves(child))
    return leaves

In [29]:
ROOT_URL + get_all_leaves(articles)[0]

'https://en.wikipedia.org//wiki/Julie_Andrews'

In [30]:
def get_articles_by_category(articles):
    categories = {}
    for category in articles['children']:
        categories[category['name']] = get_all_leaves(category)
    return categories

In [31]:
get_articles_by_category(articles)

{'People': ['/wiki/Julie_Andrews',
  '/wiki/Lauren_Bacall',
  '/wiki/Amitabh_Bachchan',
  '/wiki/Brigitte_Bardot',
  '/wiki/Ingrid_Bergman',
  '/wiki/Sarah_Bernhardt',
  '/wiki/Humphrey_Bogart',
  '/wiki/Marlon_Brando',
  '/wiki/James_Cagney',
  '/wiki/Jackie_Chan',
  '/wiki/Claudette_Colbert',
  '/wiki/Sean_Connery',
  '/wiki/Gary_Cooper',
  '/wiki/Joan_Crawford',
  '/wiki/Bette_Davis',
  '/wiki/Doris_Day',
  '/wiki/Robert_De_Niro',
  '/wiki/Alain_Delon',
  '/wiki/Marlene_Dietrich',
  '/wiki/Clint_Eastwood',
  '/wiki/Henry_Fonda',
  '/wiki/Jean_Gabin',
  '/wiki/Clark_Gable',
  '/wiki/Greta_Garbo',
  '/wiki/Judy_Garland',
  '/wiki/John_Gielgud',
  '/wiki/Lillian_Gish',
  '/wiki/Cary_Grant',
  '/wiki/Alec_Guinness',
  '/wiki/Tom_Hanks',
  '/wiki/Setsuko_Hara',
  '/wiki/Rita_Hayworth',
  '/wiki/Audrey_Hepburn',
  '/wiki/Katharine_Hepburn',
  '/wiki/Grace_Kelly',
  '/wiki/Klaus_Kinski',
  '/wiki/Vivien_Leigh',
  '/wiki/Sophia_Loren',
  '/wiki/Madhubala',
  '/wiki/Marcello_Mastroianni',
  

In [73]:
def make_features(data):
    features = []
    i = 0
    try:
        for label, url_paths in data.items():
            for url_path in url_paths:
                i += 1
                if i == 100:
                    return features
                time.sleep(0.1)
                request = requests.get(ROOT_URL + url_path)
                if request.status_code == 200:
                    html = request.content
                    soup = bs4.BeautifulSoup(html, 'html.parser')
                    text = soup.get_text()
                    tokens = nltk.tokenize.word_tokenize(text)
                    tokens = [token for token in tokens if token not in stop_words]
                    tokens = [token for token in tokens if not any([(punc in token) for punc in string.punctuation])]
                    word_features = {token:True for token in set(tokens)}
                    features.append((word_features, label))
                else:
                    print(f'Bad Request ({request.status_code})')
    except:
        print('error')
        print(len(features))
    finally:
        return features

In [74]:
category_features = make_features(get_articles_by_category(articles))

error
52


In [75]:
category_features[0]

({'Leonardis': True,
  'Merle': True,
  'Hayley': True,
  'California': True,
  'Clown': True,
  'Big': True,
  'PLWABN': True,
  'intensely': True,
  'MelayuNederlands日本語Norsk': True,
  'Perry': True,
  'Heads': True,
  'Lightnin': True,
  'revenge': True,
  'maternal': True,
  'Not': True,
  'Thomas': True,
  'libraries': True,
  'Q': True,
  'circuit': True,
  '2022': True,
  'Paltrow': True,
  'Screen': True,
  '2010s': True,
  'memoir': True,
  'asked': True,
  'He': True,
  'Two': True,
  'Ira': True,
  'About': True,
  'Reel': True,
  'prerecorded': True,
  'concurrent': True,
  'apply': True,
  'went': True,
  'Sydow': True,
  'Laurel': True,
  'Photographic': True,
  'call': True,
  'Haffkine': True,
  'Ginger': True,
  'Giullian': True,
  'Logan': True,
  'Running': True,
  'Silverstein': True,
  'Disclaimers': True,
  'portalRecent': True,
  '1962': True,
  'voice': True,
  'tributes': True,
  'Other': True,
  'Loudon': True,
  'Sterling': True,
  'Emmylou': True,
  'moving'

In [ ]:
with open('features.pkl','wb') as outfile:
    pickle.dump(category_features, outfile)

In [ ]:
with open('features.pkl','rb') as infile:
    category_features = pickle.load(infile)

In [76]:
classifier = nltk.classify.DecisionTreeClassifier.train(category_features, binary=True)

In [80]:
classifier.show_most_informative_features(5)

AttributeError: 'DecisionTreeClassifier' object has no attribute 'show_most_informative_features'

In [82]:
dir(classifier)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_decisions',
 '_default',
 '_fname',
 '_label',
 'best_binary_stump',
 'best_stump',
 'binary_stump',
 'classify',
 'classify_many',
 'error',
 'labels',
 'leaf',
 'pretty_format',
 'prob_classify',
 'prob_classify_many',
 'pseudocode',
 'refine',
 'stump',
 'train']

In [ ]:
def split_features(features, word):
    features_in = []
    features_out = []
    for label, words in fetures:
        if word in words: features_in.append((label, words))
        else: features_in.append((label, words))
    return features_in, features_out